In [1]:
pip install pandas mysql-connector-python tk

In [ ]:
import mysql.connector
from tkinter import *
from tkinter import ttk, messagebox

# Connect to MySQL Database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Sandy2208',
    database='attendance_db'
)
cursor = conn.cursor()

# Create Table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS attendance (
    id INT AUTO_INCREMENT PRIMARY KEY,
    student_name VARCHAR(255) NOT NULL,
    date DATE NOT NULL,
    status ENUM('Present', 'Absent') NOT NULL
)
""")
conn.commit()

def mark_attendance():
    name = name_entry.get()
    date = date_entry.get()
    status = status_var.get()
    if name and date and status:
        cursor.execute("INSERT INTO attendance (student_name, date, status) VALUES (%s, %s, %s)", (name, date, status))
        conn.commit()
        messagebox.showinfo("Success", "Attendance Marked Successfully")
        view_attendance()
    else:
        messagebox.showwarning("Warning", "Please fill all fields")

def view_attendance():
    cursor.execute("SELECT * FROM attendance")
    records = cursor.fetchall()
    attendance_list.delete(*attendance_list.get_children())
    for row in records:
        attendance_list.insert("", "end", values=row)

# GUI Application
root = Tk()
root.title("Attendance Tracker")
root.geometry("500x400")

Label(root, text="Student Name:").pack()
name_entry = Entry(root)
name_entry.pack()

Label(root, text="Date (YYYY-MM-DD):").pack()
date_entry = Entry(root)
date_entry.pack()

Label(root, text="Status:").pack()
status_var = StringVar()
status_dropdown = ttk.Combobox(root, textvariable=status_var, values=("Present", "Absent"))
status_dropdown.pack()
status_dropdown.current(0)

Button(root, text="Mark Attendance", command=mark_attendance).pack()

attendance_list = ttk.Treeview(root, columns=("ID", "Name", "Date", "Status"), show="headings")
attendance_list.heading("ID", text="ID")
attendance_list.heading("Name", text="Student Name")
attendance_list.heading("Date", text="Date")
attendance_list.heading("Status", text="Status")
attendance_list.pack(fill=BOTH, expand=True)

view_attendance()
root.mainloop()

# Close connection on exit
conn.close()
